In [11]:
from tokencost import calculate_prompt_cost, calculate_completion_cost

model = "gpt-4-turbo"
prompt = [{ "role": "user", "content": '''You are a helpful assistant with access to this API:
            {
                "name": "book_flight",
                "description": "Book a flight from one location to another on particular date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start": {
                            "type": "string",
                            "description": "Where the user wants to travel from"
                        },
                        "destination": {
                            "type": "string",
                            "description": "Where the user wants to travel to."
                        }
                        "date": {
                            "type": "string",
                            "description": "Date of the flight"
                        }
                    },
                    "required": [
                        "start",
                        "destination",
                        "date"
                    ]
                }
            }
          Ask the user for all required parameters or return the API call.
          User: Hi my name is Krystian. I need to fly to Miami on 14.02.2024. Can you book a flight for me?'''
           }
           ]
completion = '''<function_call>
                book_flight
                {
                "start": "Paris",
                "destination": "Miami",
                "date": "12/04/2028"
                }
                </function_call>'''

prompt_cost = calculate_prompt_cost(prompt, model)
completion_cost = calculate_completion_cost(completion, model)

print(f"{prompt_cost} + {completion_cost} = {(prompt_cost + completion_cost)}")

0.00213 + 0.00129 = 0.00342
